# 📌 상품 정보 크롤링  
* urllib 라이브러리 사용해 파일 다운로드

### ✏️ 상품 페이지 목록 받아오기

In [ ]:
# 라이브러리 읽어들이기
import requests
import re
import os
from bs4 import BeautifulSoup as bs

# 파일 주소 크롤링
def bs_parsing(url):
  response = requests.get(root_url + url)
  soup = bs(response.text, "html.parser")
  elements = soup.select("table#flisttable td > a")
  
  # 주소 담을 리스트
  page_list = []
  
  # 상세 페이지 주소 리스트화
  for (idx, el) in enumerate(elements, 1):
    page_list.append(root_url + el.attrs["href"].replace("%2F", "/"))
    
  return page_list

# 리스트 값을 txt 파일에 한 줄씩 저장
def save_text_file(page_list):
  f = open("C:/Users/sodud/study/ssafy_spec_pjt/subtitle/data/page_url_list.txt", "w", encoding="UTF8")
  for page in page_list:
    f.write(page)
    f.write("\n")
  f.close
  ("[SUCCESS] 자막 페이지 목록 저장")

# 자막 사이트
root_url = "https://kitsunekko.net"

# 일본어 자막 목록 페이지
page_url = "/dirlist.php?dir=subtitles/japanese/"

# 자막 페이지 목록 리스트로 저장
page_list = bs_parsing(page_url)

# 리스트를 txt 파일로 저장
save_text_file(page_list)

### ✏️ 하위 카테고리 별 링크 저장

In [98]:
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
import requests
import re
import os
from selenium import webdriver
from bs4 import BeautifulSoup

# 주소 담을 리스트
page_list = []

# small_categories.txt 파일 읽어오기
small_categories = "C:\SSAFY\자율프로젝트\자율크롤링\small_categories.txt"
f = open(small_categories, "r", encoding="UTF8")


while True:
    line = f.readline()
#     print(line)
    if not line: break
    strings = line.split()
    category_uid = strings[0]
    category_url = strings[1]
    print(category_uid)
    print(category_url)
    
    ## 크롤링
    webdriver_options=webdriver.ChromeOptions()
    webdriver_options.add_argument("--start-maximized")
    web_driver = webdriver.Chrome(options=webdriver_options)
    web_driver.get(category_url)
    web_driver.implicitly_wait(5)
    html = web_driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    link_list = soup.find_all('a', class_='srchGridProductUnitLink',href=True)
    
    # print(link_list)
    # print(len(link_list))

#     목록에서 link 얻어오기 (2개씩 얻어와지기때문에 짝수번은 날려보내기)
    for index, list in enumerate(link_list):
        if index %2 == 0:
            page_list.append(category_uid + " " + list['href'])
    
    
    # page_list 저장하기 
    fo = open("C:\SSAFY\자율프로젝트\자율크롤링\products.txt", "w", encoding="UTF8")
    for page in page_list:
        fo.write(page)
        fo.write("\n")
    fo.close
    
f.close


    


1
https://www.lotteon.com/search/render/render.ecn?render=nqapi&platform=pc&collection_id=9&u9=navigate&u8=FC01200000&login=Y&mallId=1
2
https://www.lotteon.com/search/render/render.ecn?render=nqapi&platform=pc&collection_id=9&login=Y&u9=navigate&u8=FC01260000&mallId=1
3
https://www.lotteon.com/search/render/render.ecn?render=nqapi&platform=pc&collection_id=9&u9=navigate&u8=FC01080000&login=Y&mallId=1
4
https://www.lotteon.com/search/render/render.ecn?render=nqapi&platform=pc&collection_id=9&login=Y&u9=navigate&u8=FC01040000&mallId=1
5
https://www.lotteon.com/search/render/render.ecn?render=nqapi&platform=pc&collection_id=9&login=Y&u9=navigate&u8=FC01220000&mallId=1
6
https://www.lotteon.com/search/render/render.ecn?render=nqapi&platform=pc&collection_id=9&login=Y&u9=navigate&u8=FC02080000&mallId=1
7
https://www.lotteon.com/search/render/render.ecn?render=nqapi&platform=pc&collection_id=9&login=Y&u9=navigate&u8=FC02090000&mallId=1
8
https://www.lotteon.com/search/render/render.ecn?rend

<function TextIOWrapper.close()>

### ✏️ 상품 정보 DB에 저장

In [115]:
# import chromedriver_autoinstaller
# chromedriver_autoinstaller.install()

import requests
import re
import os
from selenium import webdriver
from bs4 import BeautifulSoup
from openpyxl import Workbook
import pymysql

# DB 연결용
# conn = pymysql.connect(host='k7e201.p.ssafy.io',
#                        user='abuyz7e201',
#                        password='a-buy-z-e201',
#                        db='wkdbf',
#                        charset='utf8')

def connectiondb():
    conn = pymysql.connect(host='k7e201.p.ssafy.io',
                       user='abuyz7e201',
                       password='a-buy-z-e201',
                       db='wkdbf',
                       charset='utf8')
    return conn

# small_categories.txt 파일 읽어오기
products = "C:\SSAFY\자율프로젝트\자율크롤링\products.txt"
f = open(products, "r", encoding="UTF8")


while True:
    line = f.readline()
#     print(line)
    if not line: break
    strings = line.split()
    category_uid = strings[0]
    url = strings[1]
    print(category_uid)
    print(url)
    print("------------------------")

    # 임시 url
    # temp_url= "https://www.lotteon.com/p/product/LO1987818462?areaCode=AD_CPC&entryPoint=ad&dp_infw_cd=CASFC01200000&clickId=C33295679634"
#     temp_url = "https://www.lotteon.com/p/product/LO1566256308?areaCode=AD_CPC&entryPoint=ad&dp_infw_cd=CASFC04020000&clickId=C33403790992"

    webdriver_options=webdriver.ChromeOptions()
    webdriver_options.add_argument("--start-maximized")
    web_driver = webdriver.Chrome(options=webdriver_options)
    web_driver.get(url)
    web_driver.implicitly_wait(3)
    html = web_driver.page_source
    soup = BeautifulSoup(html, 'html.parser')


    # 데이터 찾기 
    product_name = soup.find('div', class_='productName').text
    des_picture = soup.find('iframe', class_='m2-prd-frame')['src']


    # origin = soup.find('div', class_='productName').text
    # status = soup.find('div', class_='productName').text
    # producer = soup.find('div', class_='productName').text
    status = '새상품'

    ## 할인율 없으면
    if soup.find_all("span", {"class": "discountRate"}):
        discount_rate = soup.find('span', class_='discountRate').text.replace("할인율","").replace(" ","").replace("%","")
        product_price = soup.find('div', class_='listPrice').select_one(":nth-child(2)").text.replace(" ","").replace("\n","").replace(",","")
    else:
        discount_rate = 0
        product_price = soup.find('div', class_='price').text.replace(" ","").replace("\n","").replace("할인가","").replace("원","").replace(",","")

    # int값으로 변환
    product_price = int(product_price)
    discount_rate = int(discount_rate)

    # 리뷰가 있는지 확인
    if soup.find_all("a", {"class": "staring"}):
        review_rating = soup.find('a', class_='staring').em.text # 5.0
    else:
        review_rating = 0;
    delivery_fee = soup.find_all('li', class_='soonInfoList')[1].text

    # 배송비 무료 > int 변환
    val = "무료" in delivery_fee
    if val:
        delivery_fee = 0
    else:
        delivery_fee = 3000



    # 이미지(썸네일) 
    picture_list = soup.find_all('div', class_='productVisualThumbs')
    
    print(picture_list)
    
#     rep_img = picture_list[1]['src']
    
#     min_length = min(len(picture_list),3)
#     print("len",len(picture_list))
    
    # 최소 1개 최대 3개 
#     max = 0
#     if (len(picture_list) <=3):
#         max = 1
#         product_thumbnail_1 = picture_list[1]['src']
#         if (len(picture_list) == 2):
#             max = 2
#             product_thumbnail_2 = picture_list[2]['src']
#         if (len(picture_list) == 3):
#             max = 3
#             product_thumbnail_2 = picture_list[2]['src']
#             product_thumbnail_3 = picture_list[3]['src']
#     else: # picture_list가 3개보다 많을때
#         max = 3
#         product_thumbnail_1 = picture_list[1]['src']
#         product_thumbnail_2 = picture_list[2]['src']
#         product_thumbnail_3 = picture_list[3]['src']

    ############### 브랜드 ################
#     brand_uid = 0
#     brand_isAlready = True
#     if soup.find_all("div", {"class": "brand"}):
#         brand_name = soup.find("div", {"class": "brand"}).text
#         brand_url = soup.find("div", {"class": "brandImage"}).select_one(":nth-child(1)")['src']
#         print(brand_name)
#         # 해당 브랜드를 가지고 있는지 확인
#         sql = "SELECT uid FROM brands WHERE name = %s"
#         with connectiondb() as conn:
#             with conn.cursor() as cur:
#                 cur.execute(sql, brand_name)
#                 result = cur.fetchone()
#                 if result is None:
#                     print("table에 브랜드없음")
#                     brand_isAlready = False
#                 else: 
#                     print("table에 브랜드있음")
#                     brand_isAlready = True
#                     brand_uid = result[0]
#     else:
#         print("브랜드없는상품")
#         brand_uid =  1 # 브랜드 없음 


#     if(brand_isAlready is False):
#         print("브랜드생성")
#         # 없으면 새로 만들고 uid 값 가져오기
#         sql = "INSERT INTO brands (img_url, name) VALUES (%s, %s)"
#         with connectiondb() as conn:
#             with conn.cursor() as cur:
#                 cur.execute(sql, (brand_url, brand_name))
#                 brand_uid = cur.lastrowid
#                 conn.commit()


    ############### product insert ################
#     print("delivery_fee",delivery_fee)
#     print("discount_rate",discount_rate)
#     print("price",product_price)
#     print("name",product_name)
#     print("description_img",des_picture)
#     print("rep_img",rep_img)
#     print("review_rate",review_rating)
#     print("status",status)
#     print("brand_uid",brand_uid)
#     print("small_categories_uid", category_uid)

    print("====================================================")


#     sql = "INSERT INTO products (delivery_fee, description_img,discount_rate,name, price, rep_img, review_rate, status, brands_uid, small_categories_uid)"
#         +" VALUES (%d,%s,%d,%d,%s,%s,%f,%s,%d,%d)"
#     with connectiondb() as conn:
#         with conn.cursor() as cur:
#             cur.execute(sql, (delivery_fee, des_picture, discount_rate, product_name, product_price, product_thumbnail_1,review_rating, status, brand_uid, small_categories_uid))
#             product_uid = cur.lastrowid
#             conn.commit()


    ############### 이미지 insert ################
    ## insert 이후 가장 최근 product uid 데려와서 상품 사진 테이블에 insert 하기 
    
#     for i in range(min_length):
#         product_thumbnail = picture_list[i]['src']
    
#     sql = "INSERT INTO products (img_url,products_uid)"
#         +" VALUES (%s,%s)"
#     with connectiondb() as conn:
#         with conn.cursor() as cur:
#             cur.execute(sql, (product_uid))
#             conn.commit()
    
    
    

f.close


1
https://displayad.lotteon.com/cadClick?x=C%2B02hjAxYkDjNxM3xVRXefXVyQG%2FBbyHJolebvAUerhSanqgoGZtOQzp3jFQeISA3sZXVFATBpg3VtnQPVzhRl60ms6CAYfrMTs63BCjguFp1fBOfmFf5KneRI4cx%2BErCqe%2FLSihysLAe1CePVNWkFYp3P6sGMiE2V9Y3VEWXCpb9aA%2BGBxtnnO41RmQMuBCaERxwLT9vXiPvQeYVjnSFEkGCGeMFCQbu%2FmqVje%2BKkM5ta0GsG7JMk7JoeDeofM%2BbY23VEVIKJEJqBMr6SjGlQ%3D%3D&fwurl=https%3A%2F%2Fwww.lotteon.com%2Fp%2Fproduct%2FLO1990862107%3FareaCode%3DAD_CPC%26entryPoint%3Dad%26dp_infw_cd%3DCASFC01200000
------------------------
[<div class="swiper-container productVisualThumbs swiper-container-initialized swiper-container-vertical" data-v-02fae411=""> <div class="swiper-wrapper" style="transition-duration: 0ms; transform: translate3d(0px, 159px, 0px);"><div class="swiper-slide swiper-slide-active" data-v-02fae411="" style="margin-bottom: 10px;"><div class="imgWrap" data-cmpnt-name="pdd_thumb_preview_img" data-cmpnt-typ="pdd_basicinfo" data-gtm-vis-first-on-screen-11901096_2002="902" data-gtm-vis-first-on-screen-119010

Exception ignored in: <function Service.__del__ at 0x000001D0E70A2700>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 170, in __del__
    self.stop()
  File "C:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 145, in stop
    self.send_remote_shutdown_command()
  File "C:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 127, in send_remote_shutdown_command
    if not self.is_connectable():
  File "C:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 116, in is_connectable
    return utils.is_connectable(self.port)
  File "C:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\common\utils.py", line 107, in is_connectable
    socket_ = socket.create_connection((host, port), 1)
  File "C:\ProgramData\Anaconda3\lib\socket.py", line 832, in create_connection
    sock.connect(sa)
KeyboardInterrupt: 

[<div class="swiper-container productVisualThumbs swiper-container-initialized swiper-container-vertical" data-v-02fae411=""> <div class="swiper-wrapper" style="transition-duration: 0ms; transform: translate3d(0px, 159px, 0px);"><div class="swiper-slide swiper-slide-active" data-v-02fae411="" style="margin-bottom: 10px;"><div class="imgWrap" data-cmpnt-name="pdd_thumb_preview_img" data-cmpnt-typ="pdd_basicinfo" data-gtm-vis-first-on-screen-11901096_2002="926" data-gtm-vis-first-on-screen-11901096_2006="962" data-gtm-vis-has-fired-11901096_2002="1" data-gtm-vis-has-fired-11901096_2006="1" data-gtm-vis-recent-on-screen-11901096_2002="926" data-gtm-vis-recent-on-screen-11901096_2006="962" data-gtm-vis-total-visible-time-11901096_2002="100" data-gtm-vis-total-visible-time-11901096_2006="100" data-v-02fae411=""><img alt="[경북농협] 자연과 함께 자란 문경농협 감홍사과 3kg, 5kg" class="loaded" data-v-02fae411="" src="https://contents.lotteon.com/itemimage/_v161938/LO/19/90/74/62/01/_1/99/07/46/20/2/LO1990746201_

[<div class="swiper-container productVisualThumbs swiper-container-initialized swiper-container-vertical" data-v-02fae411=""> <div class="swiper-wrapper" style="transition-duration: 0ms; transform: translate3d(0px, 159px, 0px);"><div class="swiper-slide swiper-slide-active" data-v-02fae411="" style="margin-bottom: 10px;"><div class="imgWrap" data-cmpnt-name="pdd_thumb_preview_img" data-cmpnt-typ="pdd_basicinfo" data-gtm-vis-first-on-screen-11901096_2002="918" data-gtm-vis-first-on-screen-11901096_2006="955" data-gtm-vis-has-fired-11901096_2002="1" data-gtm-vis-has-fired-11901096_2006="1" data-gtm-vis-recent-on-screen-11901096_2002="918" data-gtm-vis-recent-on-screen-11901096_2006="955" data-gtm-vis-total-visible-time-11901096_2002="100" data-gtm-vis-total-visible-time-11901096_2006="100" data-v-02fae411=""><img alt="무농약 제주 노지감귤 고당도 노지 귤 조생귤 밀감 5kg 10kg" class="loaded" data-v-02fae411="" src="https://contents.lotteon.com/itemimage/_v224956/LO/19/88/01/74/01/_1/98/80/17/40/2/LO1988017401

[<div class="swiper-container productVisualThumbs swiper-container-initialized swiper-container-vertical" data-v-02fae411=""> <div class="swiper-wrapper" style="transition-duration: 0ms; transform: translate3d(0px, 159px, 0px);"><div class="swiper-slide swiper-slide-active" data-v-02fae411="" style="margin-bottom: 10px;"><div class="imgWrap" data-cmpnt-name="pdd_thumb_preview_img" data-cmpnt-typ="pdd_basicinfo" data-gtm-vis-first-on-screen-11901096_2002="919" data-gtm-vis-first-on-screen-11901096_2006="936" data-gtm-vis-has-fired-11901096_2002="1" data-gtm-vis-has-fired-11901096_2006="1" data-gtm-vis-recent-on-screen-11901096_2002="919" data-gtm-vis-recent-on-screen-11901096_2006="936" data-gtm-vis-total-visible-time-11901096_2002="100" data-gtm-vis-total-visible-time-11901096_2006="100" data-v-02fae411=""><img alt="[프레시박스] GAP 경북 예천 사과 3kg 구성 부사 " class="loaded" data-v-02fae411="" src="https://contents.lotteon.com/itemimage/_v175409/LO/19/90/80/48/84/_1/99/08/04/88/5/LO1990804884_1990

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=106.0.5249.119)
Stacktrace:
Backtrace:
	Ordinal0 [0x00C61353+2233171]
	Ordinal0 [0x00BF8E31+1805873]
	Ordinal0 [0x00B066FD+812797]
	Ordinal0 [0x00AEDFFA+712698]
	Ordinal0 [0x00B5506B+1134699]
	Ordinal0 [0x00B6514A+1200458]
	Ordinal0 [0x00B518A6+1120422]
	Ordinal0 [0x00B2A73D+960317]
	Ordinal0 [0x00B2B71F+964383]
	GetHandleVerifier [0x00F0A2C2+2728322]
	GetHandleVerifier [0x00EFC184+2670660]
	GetHandleVerifier [0x00CF102A+528106]
	GetHandleVerifier [0x00CEFE10+523472]
	Ordinal0 [0x00C0028C+1835660]
	Ordinal0 [0x00C04758+1853272]
	Ordinal0 [0x00C04845+1853509]
	Ordinal0 [0x00C0E7E4+1894372]
	BaseThreadInitThunk [0x7653FA29+25]
	RtlGetAppContainerNamedObjectPath [0x775E7A7E+286]
	RtlGetAppContainerNamedObjectPath [0x775E7A4E+238]
